In [5]:
import ctypes
import numpy as np


In [13]:
# Load the shared library
lib = ctypes.CDLL('../psublast/libkarlin.so')  # For Windows use '.dll' instead of '.so'



# Define the C function prototype
lib.karlin.argtypes = [
    ctypes.c_int,                   # low (int)
    ctypes.c_int,                   # high (int)
    ctypes.POINTER(ctypes.c_double),# pr (double *)
    ctypes.POINTER(ctypes.c_double),# lambda (double *)
    ctypes.POINTER(ctypes.c_double) # K (double *)
]

lib.karlin.restype = ctypes.c_int    # Return type int

# Python function to call the C function
def call_karlin(low, high, pr_array):
    # Convert pr_array to a ctypes array
    pr = (ctypes.c_double * len(pr_array))(*pr_array)
    
    # Prepare ctypes variables to hold lambda and K
    lambda_val = ctypes.c_double()
    K_val = ctypes.c_double()
    
    # Call the C function
    result = lib.karlin(low, high, pr, ctypes.byref(lambda_val), ctypes.byref(K_val))
    
    if result != 1:
        raise RuntimeError("C function failed")
    
    # Return the computed lambda and K
    return lambda_val.value, K_val.value

# Example usage
low = -2
high = 3
pr_array = np.array([0.7, 0.0, 0.1, 0.0, 0.0, 0.2])  # Example probabilities
lambda_val, K_val = call_karlin(low, high, pr_array)

print(f"Lambda: {lambda_val}, K: {K_val}")

Lambda: 0.3299519121646881, K: 0.15372614418394498
